# Projeto 2 - Ciência dos Dados

Nome: André Nadalini

Nome: João Zsigmond

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [ ]:
%%capture

#Instalando o tweepy

In [3]:
# import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
from random import randint
import re

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [ ]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [ ]:
#Produto escolhido:
produto = 'Johnny Depp'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [4]:
data = pd.read_excel("apple.xlsx")

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
data.rename(columns={'Unnamed: 1': 'Relevância'}, inplace=True)

Comecamos limpando a base de dados. Removemos símbolos, pontuações, e characteres que não são imprimiveis (tab, quebras de linha, return, etc)

In [6]:
index = 0

clean_data = []

for tweet, r in zip(data['Treinamento'], data["Relevância"]):
    raw_data = tweet.split()
    clean_tweet = []
    for word in raw_data:
        if not word.startswith('http'):
            clean_word = re.sub('[\n:;,\'\")(@#]+', '', word)
            clean_tweet.append(clean_word)
    clean_tweet = " ".join(clean_tweet)
    clean_data.append([clean_tweet, r])
    

In [7]:
clean_data

[['deixei os fones da apple no hostel ', 1],
 ['apple eu to sedentaaaa appleevent', 2],
 ['mcfly anunciou hj retorno e show em londres dia 20/11 e hj tbm a apple anuncia o novo iphone! fico animada com as duas coisas msm sabendo que nem vou ao show e nem vou poder comprar esse iphone tão cedo! vidareal mcfly appleevent',
  2],
 ['a samsung interagindo nos tweets da apple kkkkkkkkkkkkkkkk to rindo', 1],
 ['bom dia 🙏🙏🙏', 0],
 ['apple_juice53 meu trauma com drama romântico é esse eu chego a suar frio',
  0],
 ['pamella favor passar reto', 0],
 ['yl0han du26avella o skeletonr6s tinha comprado um apple watch lá também e não chegou se não me engano',
  2],
 ['altamente recomendado...', 0],
 ['em meio a queda mundial na venda de celulares apple lança iphone 11', 2],
 ['twitter for iphone nao odiava a apple', 0],
 ['todo mundo esperando as câmeras estranhas no appleevent mas eu só tô pensando se vão conseguir aumentar o preço mais um pouco. lembrado que quando a apple sobe os preços todas as o

In [8]:
data_clean = pd.DataFrame(clean_data, columns=list("TR"))

In [9]:
data_clean

,T,R
0,deixei os fones da apple no hostel,1
1,apple eu to sedentaaaa appleevent,2
2,mcfly anunciou hj retorno e show em londres di...,2
3,a samsung interagindo nos tweets da apple kkkk...,1
4,bom dia 🙏🙏🙏,0
5,apple_juice53 meu trauma com drama romântico é...,0
6,pamella favor passar reto,0
7,yl0han du26avella o skeletonr6s tinha comprado...,2
8,altamente recomendado...,0
9,em meio a queda mundial na venda de celulares ...,2


In [10]:
irrelevante = data_clean[data_clean.R == 0]
pouco_relevante = data_clean[data_clean.R == 1]
relevante = data_clean[data_clean.R == 2]

In [11]:
# relevante

In [12]:
palavras_sem_repeticao = set()
palavras_repetidas = []

for tweet in data_clean["T"]:
    for word in tweet.split():
        palavras_sem_repeticao.add(word)
        palavras_repetidas.append(word)
N=len(palavras_repetidas)
d=len(palavras_sem_repeticao)
print('N: ', N)
print('d: ',d)

N:  8035
d:  2473


In [13]:
total_irrelevante = len(irrelevante)
total_relevante = len(relevante)
total_pouco_relevante = len(pouco_relevante)

In [14]:
print("The number of irrelevant tweets is {}".format(total_irrelevante))
print("The number of little relevant tweets is {}".format(total_pouco_relevante))
print("The number of relevant tweets is {}".format(total_relevante))

The number of irrelevant tweets is 132
The number of little relevant tweets is 225
The number of relevant tweets is 143


In [15]:
def get_series_from_df(df):
    arr = []
    for tweet in df["T"]:
        for word in tweet.split():
            arr.append(word)
    s = pd.Series(arr)
    return s
            
palavras_irrelevantes = get_series_from_df(irrelevante)
palavras_pouco_relevantes = get_series_from_df(pouco_relevante)
palavras_relevantes = get_series_from_df(relevante)

In [16]:
prob_ppr = palavras_pouco_relevantes.value_counts(True)
prob_pi = palavras_irrelevantes.value_counts(True)
prob_pr = palavras_relevantes.value_counts(True)

# Funcionamento do algoritimo de Naïve-Bayes

O funcionamento do algoritimo de Naïve - Bayes consiste em determinar qual é a probabilidade de uma palavra fazer parte de certo grupo (relevante, irrelevante, etc) e multiplicar todas essas probabilidades para cada palavra na frase (Tweet).


Ou seja:

$P(tweet|R) = 
P(Eu|R).P(gosto|R).P(muito|R).P(da|R).P(apple|R).P(e|R).P(quero|R).P(comprar|R).P(um|R).P(Iphone|R)$


Vamos denotar $Tweets relevantes$ simplesmente como $R$, para encurtar a fórmula




A fórmula completa fica então:

$P(R|Tweet) = \frac{P(Eu|R).P(gosto|R).P(muito|R).P(da|R).P(apple|R).P(e|R).P(quero|R).P(comprar|R).P(um|R).P(Iphone|R).P(R)}{P(Tweet)}$



# Smoothing de LaPlace
    O smoothing de Laplace é uma técnica usada para manter coerência no algoritimo, mesmo lidando com palavras que não aparecem na base de dados de treinamento. Sem o uso de smoothing, frases com palavras que não aparecem no banco de treinamento de certa categoria, certamente não serão classificadas em tal categoria. Isso prejudica a análise pois muitas vezes, frases podem ser de uma categoria mesmo tendo palavras novas, não antes vistas no treinamento dessa categoria.

In [17]:
#     palavras_irrelevantes

In [18]:
def prob_da_origem_da_frase_ser_ppr(tweet):
    prob_total = 1
    for word in tweet.split():
        if word not in palavras_pouco_relevantes.tolist():
            prob_total*=(1/((len(palavras_pouco_relevantes)+d+1)))
        else:
            prob_total*=prob_ppr[word]
    return prob_total * total_pouco_relevante / 500

In [19]:
def prob_da_origem_da_frase_ser_pr(tweet):
    prob_total = 1
    for word in tweet.split():
        if word not in palavras_relevantes.tolist():
            prob_total*=(1/((len(palavras_relevantes)+d+1)))
        else:
            prob_total*=prob_pr[word]
    return prob_total * total_relevante / 500

In [20]:
def prob_da_origem_da_frase_ser_pi(tweet):
    prob_total = 1
    for word in tweet.split():
        if word not in palavras_irrelevantes.tolist():
            prob_total*=(1/(len(palavras_irrelevantes)+d+1))
        else:
            prob_total*=prob_pi[word]
    return prob_total * total_irrelevante / 500

In [21]:
def origem_do_tweet(tweet):
    origem_ppr = prob_da_origem_da_frase_ser_ppr(tweet)
    origem_pr = prob_da_origem_da_frase_ser_pr(tweet)
    origem_pi = prob_da_origem_da_frase_ser_pi(tweet)
    
    if origem_ppr > origem_pr and origem_ppr > origem_pi:
#         print("O Tweet tem a maior probabilidade de ser pouco relevante e essa probabilidade é {}".format(origem_ppr))
        return(1)
    elif origem_pr > origem_ppr and origem_pr > origem_pi:
#         print("O Tweet tem a maior probabilidade de ser relevante e essa probabilidade é {}".format(origem_pr))
        return(2)
    else:
#         print("O Tweet tem a maior probabilidade de ser irrelevante e essa probabilidade é {}".format(origem_pi))
        return(0)
    

In [22]:
print(prob_da_origem_da_frase_ser_pi("gfgfdgffgd "))
print(prob_da_origem_da_frase_ser_ppr("fgdgdfgdf"))
print(prob_da_origem_da_frase_ser_pr("gfdfgfdfd"))
origem_do_tweet('samsung eh melhor que apple')

6.601650412603151e-05
7.521310379408325e-05
5.223744292237443e-05


2

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [23]:
teste = pd.read_excel("apple.xlsx", sheet_name="Teste")

In [24]:
teste["performance"] = 0
def seed_performance(df):
    for i in range(len(df)):
        df.loc[i, "performance"] = origem_do_tweet(df.Teste[i])
#         teste.loc[i, "performance"] = randint(0, 2)
    return df

In [25]:
seed_performance(teste)

,Teste,Unnamed: 1,performance
0,"falando em amanhã, to curiosa pra esse evento ...",2,2
1,@apple é serio assim tao dificil colocar live ...,2,0
2,main vocal do putinhas aborteiras https://t.c...,0,0
3,"amo essa música, a mensagem, a hailee, cadelaa...",0,0
4,é triste a vida do pobre fanboy da apple. acab...,1,1
5,cada vez q ouço essa ela melhora https://t.co/...,0,0
6,@apple @samsungbrasil é melhor que apple contr...,2,2
7,@yllabiavatti esse é o apple watch mesmo e olh...,2,2
8,"▶ ouça pelo site https://t.co/fxqenmzgct, pelo...",0,0
9,a charli deu tudo que os gays queriam 🥺❤️ http...,0,0


In [29]:
def get_performance(df):
    relevante_verdadeiro = 0
    relevante_falso = 0
    pouco_relevante_verdadeiro = 0
    pouco_relevante_falso = 0
    irrelevante_verdadeiro = 0
    irrelevante_falso = 0
    for i in range(len(df)):
        if (df.loc[i, "performance"] == 2 and df.loc[i, "Unnamed: 1"] == 2):
            relevante_verdadeiro += 1
        elif (df.loc[i, "performance"] == 2 and df.loc[i, "Unnamed: 1"] != 2):
            relevante_falso += 1
        elif (df.loc[i, "performance"] == 1 and df.loc[i, "Unnamed: 1"] == 1):
            pouco_relevante_verdadeiro += 1
        elif (df.loc[i, "performance"] == 1 and df.loc[i, "Unnamed: 1"] != 1):
            pouco_relevante_falso += 1
        elif (df.loc[i, "performance"] == 0 and df.loc[i, "Unnamed: 1"] == 0):
            irrelevante_verdadeiro += 1
        elif (df.loc[i, "performance"] == 0 and df.loc[i, "Unnamed: 1"] != 0):
            irrelevante_falso += 1
    print("porcentagem de relevante verdadeiro: {}".format(relevante_verdadeiro/250))  
    print("porcentagem de relevante falso: {}".format(relevante_falso/250))  
    print("porcentagem de pouco relevante verdadeiro: {}".format(pouco_relevante_verdadeiro/250))  
    print("porcentagem de pouco relevante falso: {}".format(pouco_relevante_falso/250))  
    print("porcentagem de irrelevante verdadeiro: {}".format(irrelevante_verdadeiro/250))  
    print("porcentagem de irrelevante falso: {}".format(irrelevante_falso/250))
    
    acertos = [relevante_verdadeiro, pouco_relevante_verdadeiro, irrelevante_verdadeiro]
    print("A porcentagem de acerto de nosso algoritimo é de {0}%".format(sum(acertos)/250))
        

In [30]:
get_performance(teste)

porcentagem de relevante verdadeiro: 0.092
porcentagem de relevante falso: 0.216
porcentagem de pouco relevante verdadeiro: 0.248
porcentagem de pouco relevante falso: 0.136
porcentagem de irrelevante verdadeiro: 0.196
porcentagem de irrelevante falso: 0.108
A porcentagem de acerto de nosso algoritimo é de 0.536%


In [28]:
def saber_se_tem_algum_grau_de_relevancia(df):
    soma = 0
    for i in range(len(df)):
        if (df.loc[i, "performance"] != 0 and df.loc[i, "Unnamed: 1"] != 0):
            soma += 1
        if (df.loc[i, "performance"] == 0 and df.loc[i, "Unnamed: 1"] == 0):
            soma += 1
    print(soma / 250)
saber_se_tem_algum_grau_de_relevancia(teste)

0.704


___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

## Porque não podemos usar nosso classificador para gerar mais amostras de treinamento

Não podemos usar nosso classificador para gerar mais amostras de treinamento porque ele aprende a classificar com um input externo, que nesse caso é a informação que passamos a ele. Se o input para ele aprender for gerado por ele mesmo, ele só ficaria concordando com si mesmo, e não haveria aprendizado.

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**